# 目錄


## [STEP01 讀檔案](#STEP1-讀檔案)


## [STEP02 Parse "FROM" Statement (Basic Logic)](#STEP2-Parse-"FROM"-Statement)


## [STEP03 Inspect Parse SQL](#STEP3-Inspect-Parse-SQL)


## [STEP04 彙整成報表 並輸出 ](#Report)




# STEP1 讀檔案

# Package


## Install Package


1. 在 Jupyter Notebook 內直接安裝 （最容易 若主機上有其他版本 也不會搞錯）


2. 使用 CMD / Command Line 安裝 （要注意安裝的位置）

In [ ]:
!pip3 install pandas

# 使用 Command Line 安裝


```shell
pip3 -V

```

`pip 20.3.3 from /usr/local/lib/python3.9/site-packages/pip (python 3.9)`


確認安裝目錄是目前的 python 版本 （上面是我本機的安裝位置 HomeBrew）



`pip3 install pandas`





`pip3 show pandas`



```
Name: pandas
Version: 1.1.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: /usr/local/lib/python3.9/site-packages
Requires: pytz, python-dateutil, numpy
Required-by: 

```



# Import Package

In [ ]:
import pathlib
import re

# 安裝好的 pandas
import pandas as pd
# 簡寫 pd 為個人習慣

pd.set_option('display.max_colwidth', None)

# Script File 在 本機的位置

In [ ]:

# Script 檔案所在資料夾
path = 'Script/DWSCRIPT/'

# 要抓取的檔案 全部都是 .txt 為副檔名
# File Pattern 
file_pattern = '*.txt'



# 檢查路徑是否正確

In [ ]:

# 將 STRING 轉換成 PATH OBJECT
p = pathlib.Path(path)

# 檢查路徑是否正確
p.exists()

# 抓檔

In [ ]:

# 路徑確定正確後
# 要來抓檔了

# 以 副檔名皆為 .txt 來抓取
files = p.glob(file_pattern)

files = [f for f in files]

for f in files:
    print(f)

# STEP2 Parse "FROM" Statement


[目錄](#目錄)

In [ ]:

# 先只讀取一個檔案

target_file = files[7]

f = open(target_file, 'r')

content = f.read()

f.close()


# 檢查是否關閉資源
f.closed

In [ ]:

# 看一下檔案是否正常

content


## 主要邏輯： 

- 理論上 寫 SQL 的人 應該會遵循這樣的寫法 

```sql

SELECT * FROM TABLENAME

-- 或是

SELECT *
FROM TABLENAME

-- 當然 也有不講武德的

SELECT *
FROM
TABLENAME 
```
~~`↑這種人 我勸他浩子尾汁`~~

***這種之後再來看***





- 我們要 Parse 的 是 FROM 後面的 TABLENAME


- --> 一行一行 來看 我們只要找到 FROM關鍵字 就好


In [ ]:
# 把 檔案內容一行一行切開
content = content.split('\n')


# 也可以在讀檔時 就一行一行切開
# 每一行仍會保留 \n
f = open(target_file, 'r')

# 改成 readlines
content_p = f.readlines()

f.close()


In [ ]:
# anonymous function Lambda

# 常搭配 map 一起使用  -->  map(FUNCTION, OBJECT)
# 讓傳入的 object 套用 傳入的 function
# 傳入的 object 必須是可迭代的 ( iterable )

# 去除 readlines() 留下的 換行符號 \n
content == list(map(lambda x: x.replace('\n', ''), content_p))

In [ ]:
for line in content:
    
    
    # 找到 FROM 關鍵字
    # 從 FROM 關鍵字後 下一個關鍵字通常就是 Table Name
    if 'FROM' in line:
        print('====================')
        mess = line[line.find('FROM'):].replace(";","").lstrip()
        print('有 FROM')
        
        print(mess)
        # FROM ${ROOTDB}.TABLES
        
        print('--------------')
        
        # 以空白切割 FROM TABLE
        table = mess.split()[1]  # 切割後第二個位置的字串
        print('Extract Table : {}'.format(table))
        
        
    

# 試著找看看  JOIN  關鍵字

In [ ]:
target_file = files[7]

# 如果有 Encoding 問題
# 換成 BIG5 OR UTF8 看看
# f = open(target_file, 'r', encoding='BIG5')

f = open(target_file, 'r', encoding='UTF-8')

content = f.read()

f.close()

content = content.split('\n')


## 試著加入 檔案的行數  可以回檔案確認是否正確

In [ ]:

# enumerate  -->  for loop 同時 return 計數
for i, line in enumerate(content):
    
    
    # 找到 FROM 關鍵字
    # 從 FROM 關鍵字後 下一個關鍵字通常就是 Table Name
    if 'JOIN' in line:
        print('====================')
        mess = line[line.find('JOIN'):].replace(";","").lstrip()
        print('第 {} 行 有 JOIN'.format(i + 1))
        
        print(mess)
        # FROM ${ROOTDB}.TABLES
        
        print('--------------')
        
        # 以空白切割 FROM TABLE
        table = mess.split()[1]  # 切割後第二個位置的字串
        print('Extract Table : {}'.format(table))

# STEP3 Inspect Parse SQL 


[目錄](#目錄)

# 1. 盤點關鍵字 邏輯


## Source Table 部分


- FROM & JOIN  的 額外狀況

    1. FROM 後面可以直接使用 逗號 來當作 INNER JOIN
    
    ```sql
    SELECT *
    FROM COUNTRY a, CITY b
    WHERE a.KEY = b.KEY
    
    ```
    
    2. FROM OR JOIN 後面還有 SUBQUERY 
    
    ```sql
    SELECT *
    FROM (   <--  這個捨棄
        SELECT *
        FROM COUNTRY a   <-- 這個才是要的
        LEFT JOIN (   <--  這個捨棄
            SELECT * 
            FROM AREA    <-- 這個才是要的
            WHERE UPDATE_TIME > 2021
        ) a
        ON a.KEY = b.KEY
        WHERE KEY = 'CONDITION'
        
    ) sub
    
    ```



## Target Table 部分

- DELETE

    1. 會有與 FROM 混雜的問題
    
    ```sql
    DELETE FROM EMPLOYEE   <-- 應該屬於 Target Table : Delete
    
    WHERE ID = 69
    ```


- DROP


- INSERT INTO


- UPDATE


- CREATE


- RENAME TABLE


- REPLACE VIEW (TD)


- COLLECT STATISTIC ON (TD)


- TRUNCATE TABLE (Non TD)



# 2. 精確判定 部分


## 使用 Regular Expression


### Python re : Regular expression operations

In [ ]:
target_file

In [ ]:
target_file = files[-3]

# 如果有 Encoding 問題
# 換成 BIG5 OR UTF8 看看
f = open(target_file, 'r', encoding='BIG5')

content = f.read()

f.close()

content = content.split('\n')

# enumerate  -->  for loop 同時 return 計數
for i, line in enumerate(content):
    
    
    # 找到 FROM 關鍵字
    # 從 FROM 關鍵字後 下一個關鍵字通常就是 Table Name
    if 'JOIN' in line:
        print('====================')
        mess = line[line.find('JOIN'):].replace(";","").lstrip()
        print('第 {} 行 有 JOIN'.format(i + 1))
        
        print(mess)
        # FROM ${ROOTDB}.TABLES
        
        print('--------------')
        
        # 以空白切割 FROM TABLE
        table = mess.split()[1]  # 切割後第二個位置的字串
        print('Extract Table : {}'.format(table))
        
# 出現 error 了

In [ ]:
line

## Regular Expression + Python 判斷式 的簡單運作 


### [也可以用線上版 來快速查看結果](https://regex101.com/)


- \b**EDW**\b   <-- 這個指 EDW 這個word 必須單獨出現 前後不可有其他 character 

character 為 **a 到 z**  和  **A 到 Z**  和  **0 到 9**  最後  **還有底線 _**


- \b**EDW**   <-- 這個指 EDW 前面不可有其他 character 


- **EDW**\b   <-- 這個指 EDW 後面不可有其他字 character 



In [ ]:
print('判斷式為 "JOIN" 在 該行 中 出現 ：')

print('該行為 ： {}'.format(line))

print('if "JOIN" in line : 結果為 {}'.format('JOIN' in line))



In [ ]:
print('加入 regular expression 判斷')

print('判斷式為 "JOIN" 在 該行 中 單獨出現 ：')

print('該行為 ： {}'.format(line))

print("if re.search(r\'\\bJOIN\\b\') : 結果為 {}".format(re.search(r'\bJOIN\b', line) is not None))


# 若 regular expression 找不到符合的 expression 則 return None

In [ ]:
re.search(r'\bJOIN\b', line) is None

In [ ]:
for i, line in enumerate(content):
    
    
    # 找到 FROM 關鍵字
    # 從 FROM 關鍵字後 下一個關鍵字通常就是 Table Name
#     if 'JOIN' in line:
    # flags = re.I  -->  IgnoreCases 忽略大小寫區別
    if re.search(r'\bJOIN\b', line):
    
        print('====================')
        mess = line[line.find('JOIN'):].replace(";","").lstrip()
        print('第 {} 行 有 JOIN'.format(i + 1))
        
        print(mess)
        # FROM ${ROOTDB}.TABLES
        
        print('--------------')
        
        # 以空白切割 FROM TABLE
        table = mess.split()[1]  # 切割後第二個位置的字串
        print('Extract Table : {}'.format(table))
        
# 沒有錯了 也把 23 行 去除了

# Report



## 彙整成報表並輸出


## 使用 Pandas 


## [額外安裝套件](#Package)



## 簡易 Pandas 報表概念


### Pandas 中最常使用的 DataFrame 可以看成 Excel 的 Sheet


<div>
<br>
<img src="excel.png" width=400px/>
</div>


### 在 Pandas 中 轉換成 List of Row

- 第一行 就是 `[100, 'Vincent']`

- 第二行 就是 `[101, 'Tim']`

- 類推


[目錄](#目錄)

In [ ]:
rows = [
    [100, 'Vincent'],  # Row 1
    [101, 'Tim'],
    [102, 'Sheldon'],
    [103, 'Scott'],
    [104, 'Tony'],
]

df = pd.DataFrame(rows, columns=['ID', 'Name'])

df

## 可看出產生報表的關鍵在 List Of Rows


- 每一行資料都可轉換成 List


- 最後再將所有 List 包成 大的 List


## 輸出報表前 可以先想好 Column 的長相



- 這次我們預估是這樣



|  檔案名稱   | Source Line  | Source Table 
|  ----  | ----  |  ----  
| File1  | 4 | PDATA.TABLENAME 
| File1  | 9 | PMART.TABLENAME 



In [ ]:
# 大的 List
report_list = []

for i, line in enumerate(content):
    
    # 找到 FROM 關鍵字
    # 從 FROM 關鍵字後 下一個關鍵字通常就是 Table Name
#     if 'JOIN' in line:
    # flags = re.I  -->  IgnoreCases 忽略大小寫區別
    if re.search(r'\bJOIN\b', line):
    
        print('====================')
        mess = line[line.find('JOIN'):].replace(";","").lstrip()
        print('第 {} 行 有 JOIN'.format(i + 1))
        
        print(mess)
        # FROM ${ROOTDB}.TABLES
        
        print('--------------')
        
        # 以空白切割 FROM TABLE
        table = mess.split()[1]  # 切割後第二個位置的字串
        print('Extract Table : {}'.format(table))
        # 有被掃出 Source Table 才要被寫入 Report 中
        # 直接把結果 裝進 Report List 中 
        report_list.append(
            [
                files[-3],
                i + 1,  # Source Lines
                table
            ]
        )
        
# 沒有錯了 也把 23 行 去除了

In [ ]:
df = pd.DataFrame(report_list, columns=['FileName', 'Source_Lines', 'Source_Table'])

df

# 報表輸出成檔案


## 寫成 excel 時 可能遇到套件問題

- pandas 本身 沒有支援 「 直接 」 讀取/寫出 excel


- 透過其他輔助套件 ＊需額外安裝 下面主要 2 個


1. openpyxl


2. xlrd


[目錄](#目錄)

In [ ]:

# 不需要 Index
df.to_csv('report.csv', index=False)


# 也可以選擇 excel

df.to_excel('report.xlsx', index=False)